In [1]:
# import onnx
import onnxruntime
import os
import numpy as np
import pandas as pd

# get current working directory
current_dir = os.getcwd()

In [2]:
# import onnx model
import onnxruntime
#import the onnx model init_model_xgb.onnx
onnx_model_path = os.path.join(current_dir, 'init_model_xgb.onnx')
onnx_model = onnxruntime.InferenceSession(onnx_model_path)

# print the model inputs and outputs
print("Model Inputs:")
for input in onnx_model.get_inputs():
    print(f"Name: {input.name}, Shape: {input.shape}, Type: {input.type}")
print("Model Outputs:")
for output in onnx_model.get_outputs():
    print(f"Name: {output.name}, Shape: {output.shape}, Type: {output.type}")



Model Inputs:
Name: input, Shape: [None, 5], Type: tensor(float)
Model Outputs:
Name: label, Shape: [None], Type: tensor(int64)
Name: probabilities, Shape: [None, 2], Type: tensor(float)


In [3]:
# load the data
data_path = os.path.join(current_dir, 'pro_10940.csv')
data = pd.read_csv(data_path)
# print the data
print("Data:")
print(data.head())

Data:
            Z0        DZ  DZDT        ECG       MXR       MYR       MZR  \
0  1523.505397 -1.345635  14.0 -42.201127 -0.919434  0.085938  0.408203   
1  1526.288696 -1.338959  28.0 -42.200173 -0.918457  0.088379  0.408691   
2  1524.417110 -1.332283  27.0 -42.209313 -0.921387  0.083984  0.407715   
3  1523.286529 -1.326084  26.0 -42.216545 -0.918945  0.089355  0.408691   
4  1523.540206 -1.319885  29.0 -42.211061 -0.916992  0.090332  0.408691   

      GyroX     GyroY     GyroZ  
0 -1.586914 -0.366211  1.159668  
1 -1.586914 -0.305176  1.037598  
2 -1.525879 -0.244141  0.976562  
3 -1.525879 -0.305176  1.037598  
4 -1.525879 -0.305176  1.098633  


In [4]:
SAMPLING_FREQUENCY = 1000;
TIME_WINDOW = 6; # in seconds
HOP_LENGTH = 512; # in number of samples
FRAME_LENGTH = 2048; # in number of samples

In [5]:
from scipy.signal import butter, filtfilt
from scipy.stats import kurtosis, skew, entropy
import librosa

def butterworth_highpass(data, cutoff_frequency=75, sampling_rate=1000, order=2):
    b, a = butter(order, cutoff_frequency, btype='high', fs=sampling_rate, analog=False)
    y = filtfilt(b, a, data)
    return y


# extract the MZR from the data into a numpy array
data_interest = data[['MZR']]

# calculate the number of windows of size 6000 in data_interest
num_windows = int(len(data_interest)/(SAMPLING_FREQUENCY* TIME_WINDOW))

# iterate over the windows and calculate the features
features = []
for i in range(num_windows):
    # get the start and end indices of the window
    start = i * SAMPLING_FREQUENCY * TIME_WINDOW
    end = start + SAMPLING_FREQUENCY * TIME_WINDOW
    # get the window
    window = data_interest.iloc[start:end].values.flatten()

    y = butterworth_highpass(window)
    y_demeaned = y - np.mean(y)

    rms = np.mean(librosa.feature.rms(y=y, frame_length=2048, hop_length=512))
    spectrum = np.fft.rfft(y)
    spectral_kurtosis = kurtosis(abs(spectrum))
    spectral_skewness = skew(abs(spectrum))
    spectral_variance = np.var(abs(spectrum))
    spectral_mean = np.sum(abs(spectrum)) / len(spectrum)

    val = [spectral_skewness, spectral_variance, spectral_kurtosis,rms,spectral_mean]
    features.append(val)

In [6]:
import numpy as np
import onnxruntime as ort

# assume 'features' is your input data with shape (batch_size, 5)

# load the ONNX model
session = ort.InferenceSession('init_model_xgb.onnx')

# run the model
outputs = session.run(['label', 'probabilities'], {'input': features})

# get the outputs
label = outputs[0]
probabilities = outputs[1]

In [7]:
print(label)

[1 1 0 0 0 0 1 1 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0
 0 0 0 0 0 0 0 1 0 0 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 0 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1
 1 1 1 1 0 0 1 0 0 1 1 1 0 1 0 0 0 0 0 0 0 0 1 0 0 1 1 1 1 1 1 1 0 0 0 0 0
 0 0 0 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 1
 1 1 1 1 1 1 1 1 1 0 0 0 1 0 1 1 0 1 1 1 1 1 1 1 1 1 0 1 0 1 0 1 1 1 1 1 1
 1 1 1 1 1 1 0 0 0 0 0 0 0 1 0 1 1 1 1 1 1 1 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1
 1 0 0 0 0 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 0 0 1 1 1 1 1 